# Installating R on WinPython

#### This procedure applys for Winpython  (Version of December 2015 and after) 
### 1 - Downloading and Installing R binary in the right place

In [1]:
import os
import sys
import io

In [2]:
os.environ["tmp_Rdirectory"]= "R" 
os.environ["R_HOME"] = os.environ["WINPYDIR"]+ "\\..\\tools\\R\\" 
os.environ["R_HOMEbin"]=os.environ["R_HOME"] + "bin" 
# for installation we need this
os.environ["tmp_Rbase"]=os.path.join(os.path.split(os.environ["WINPYDIR"])[0]  , 'tools','R' )  

In [3]:
# downloading R-3.2.3 (65 380 480 Bytes, may takes a few minutes)
try:
    import urllib.request as urllib2  # Python 3
except:
    import urllib2  # Python 2
if 'amd64' in sys.version.lower():
    r_comp ='/COMPONENTS="main,x64,translations'
else:
    r_comp ='/COMPONENTS="main,i386,translations'

# specify R binary and hash
r_url = "https://cran.r-project.org/bin/windows/base/R-3.3.1-win.exe"
hashes=("104bfc769d27c3fd3c9294cc4f82e232", "b1102b40c08914bada8f3240197ac960070c7db2" )

# specify target location
r_binary = os.path.basename(r_url)
r_installer = os.environ["WINPYDIR"]+"\\..\\tools\\"+r_binary
os.environ["r_installer"] = r_installer

# Download
g = urllib2.urlopen(r_url) 
with io.open(r_installer, 'wb') as f:
    f.write(g.read())
g.close
g = None

In [4]:
#checking it's there
!dir %r_installer%

 Le volume dans le lecteur C n'a pas de nom.
 Le num‚ro de s‚rie du volume est 98F9-A53D

 R‚pertoire de C:\WinPython\basedir35\buildQt5\winpython-64bit-3.5.2.3Qt5b2\tools

04/10/2016  19:52        73ÿ566ÿ547 R-3.3.1-win.exe
               1 fichier(s)       73ÿ566ÿ547 octets
               0 R‚p(s)  30ÿ744ÿ125ÿ440 octets libres


In [5]:
# checking it's the official R-3.2.3 
import hashlib
def give_hash(of_file, with_this):
    with io.open(r_installer, 'rb') as f:
        return with_this(f.read()).hexdigest()  
print (" "*12+"MD5"+" "*(32-12-3)+" "+" "*15+"SHA-1"+" "*(40-15-5)+"\n"+"-"*32+" "+"-"*40)

print ("%s %s %s" % (give_hash(r_installer, hashlib.md5) , give_hash(r_installer, hashlib.sha1),r_installer))
assert give_hash(r_installer, hashlib.md5) == hashes[0]
assert give_hash(r_installer, hashlib.sha1) == hashes[1]

            MD5                                 SHA-1                    
-------------------------------- ----------------------------------------
104bfc769d27c3fd3c9294cc4f82e232 b1102b40c08914bada8f3240197ac960070c7db2 C:\WinPython\basedir35\buildQt5\winpython-64bit-3.5.2.3Qt5b2\scripts\..\python-3.5.2.amd64\..\tools\R-3.3.1-win.exe


In [6]:
# let's install it 
# If you are "USB life style", or multi-winpython
#   ==> CLICK the OPTION "Don't create a StartMenuFolder' <== (when it will show up)
os.environ["tmp_R_comp"]=r_comp
!start cmd /C %r_installer% /DIR=%tmp_Rbase% %tmp_R_comp%

## During Installation (if you wan't to move the R installation after)

Choose non default option "Yes (customized startup"

then after 3 screens, Select  "Don't create a Start Menu Folder"

Un-select "Create a desktop icon"

Un-select "Save version number in registery"


<img src="https://raw.githubusercontent.com/stonebig/winpython_afterdoc/master/examples/images/r_setup_unclick_shortcut.GIF">

### 2 - create a R_launcher and install irkernel

In [7]:
import os
import sys
import io
# let's create a R launcher  
r_launcher = r"""
@echo off
call %~dp0env.bat
rscript %*
"""
r_launcher_bat = os.environ["WINPYDIR"]+"\\..\\scripts\\R_launcher.bat"

# let's create a R init script
# in manual command line, you can use repos = c('http://irkernel.github.io/', getOption('repos'))
r_initialization = r"""
install.packages(c('repr', 'IRdisplay', 'stringr', 'crayon', 'pbdZMQ', 'devtools'), repos = c('http://cran.rstudio.com/', 'http://cran.rstudio.com/'))
devtools::install_github('IRkernel/IRkernel')
library('pbdZMQ')
library('repr')
library('IRkernel')
library('IRdisplay')
library('crayon')
library('stringr')
IRkernel::installspec()
"""
r_initialization_r = os.path.normpath(os.environ["WINPYDIR"]+"\\..\\scripts\\R_initialization.r")


for i in [(r_launcher,r_launcher_bat), (r_initialization, r_initialization_r)]:
    with io.open(i[1], 'w', encoding = sys.getdefaultencoding() ) as f:
        for line in i[0].splitlines():
            f.write('%s\n' %  line  )


In [2]:
#check what we are going to do 
print ("!start cmd /C %WINPYDIR%\\..\\scripts\\R_launcher.bat --no-restore --no-save  " + r_initialization_r)

!start cmd /C %WINPYDIR%\..\scripts\R_launcher.bat --no-restore --no-save  C:\WinPython\basedir35\buildQt5\winpython-64bit-3.5.2.3Qt5b2\scripts\R_initialization.r


In [ ]:
# Launch Rkernel setup
os.environ["r_initialization_r"] = r_initialization_r
!start cmd /C %WINPYDIR%\\..\\scripts\\R_launcher.bat --no-restore --no-save  %r_initialization_r%  

In [8]:
# make RKernel a movable installation with the rest of WinPython 
from winpython import utils
base_winpython = os.path.dirname(os.path.normpath(os.environ["WINPYDIR"]))
rkernel_json=(base_winpython+"\\settings\\kernels\\ir\\kernel.json")

# so we get "argv": ["{prefix}/../tools/R/bin/x64/R"
utils.patch_sourcefile(rkernel_json, base_winpython.replace("\\","/"), r'{prefix}/..', silent_mode=False)

patching  C:\WinPython\basedir35\buildQt5\winpython-64bit-3.5.2.3Qt5b2\settings\kernels\ir\kernel.json from C:/WinPython/basedir35/buildQt5/winpython-64bit-3.5.2.3Qt5b2 to {prefix}/..


### 3- Install a R package via a IPython  Kernel

In [9]:
%load_ext rpy2.ipython

#vitals: 'dplyr', 'R.utils', 'nycflights13'
# installation takes 2 minutes
%R install.packages(c('dplyr','R.utils', 'nycflights13'), repos='http://cran.rstudio.com/') 

### 4- Small demo via R magic

In [10]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [11]:
%%R
library('dplyr')
library('nycflights13') 
write.csv(flights, "flights.csv")

In [12]:
%R head(flights) 

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
1,2013,1,1,517,515,2.0,830,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400.0,5.0,15.0,1.357016e+09
2,2013,1,1,533,529,4.0,850,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416.0,5.0,29.0,1.357016e+09
3,2013,1,1,542,540,2.0,923,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089.0,5.0,40.0,1.357016e+09
4,2013,1,1,544,545,-1.0,1004,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576.0,5.0,45.0,1.357016e+09
5,2013,1,1,554,600,-6.0,812,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762.0,6.0,0.0,1.357020e+09
6,2013,1,1,554,558,-4.0,740,728,12.0,UA,1696,N39463,EWR,ORD,150.0,719.0,5.0,58.0,1.357016e+09


In [13]:
%R airports %>% mutate(dest = faa) %>% semi_join(flights) %>% head

,faa,name,lat,lon,alt,tz,dst,dest
1,IAH,George Bush Intercontinental,29.984433,-95.341442,97,-6.0,A,IAH
2,MIA,Miami Intl,25.793250,-80.290556,8,-5.0,A,MIA
3,ATL,Hartsfield Jackson Atlanta Intl,33.636719,-84.428067,1026,-5.0,A,ATL
4,ORD,Chicago Ohare Intl,41.978603,-87.904842,668,-6.0,A,ORD
5,FLL,Fort Lauderdale Hollywood Intl,26.072583,-80.152750,9,-5.0,A,FLL
6,IAD,Washington Dulles Intl,38.944533,-77.455811,313,-5.0,A,IAD


### 5 - Installing the very best of R pakages (optional, you will start to get a really big directory)


In [14]:
# essentials: 'tidyr', 'shiny', 'ggplot2', 'caret' , 'nnet'  
# remaining of Hadley Wickahm "stack" (https://github.com/rstudio)
%R install.packages(c('tidyr', 'ggplot2',  'shiny','caret' , 'nnet'), repos='https://cran.rstudio.com/') 
%R install.packages(c('knitr', 'purrr', 'readr', 'readxl'), repos='https://cran.rstudio.com/')
%R install.packages(c('rvest', 'lubridate', 'ggvis', 'readr','base64enc'), repos='https://cran.rstudio.com/')

# TRAINING = online training book http://r4ds.had.co.nz/ (or https://github.com/hadley/r4ds)

### 6 - Relaunch Jupyter Notebook to get a R kernel option
launch a new notebook of "R" type, and type in it:
    
library('dplyr')

library('nycflights13') 

head(flights)

### 9 - To Un-install  / Re-install R (or other trouble-shooting)

- launch  winpython**\tools\R\unins000.exe

- delete the directory winpython**\tools\R

- re-install
